# Nombre: Miguel González Cobaleda

# TAREA FINAL: Datos de vivienda

<p> El negocio inmobiliario es uno de los motores de la economía de España, el carácter turístico de nuesta península hace que sean muchos los inversores extranjeros que se decidan por buscar una inversión de bajo riesgos en nuestras ciudades costeras o grandes urbes. 
Muchas de estas inversiones se hacen sobre grandes bolsas de inmuebles que deben ser analizados previamente para comprobar la rentabilidad del porfolio </p>

<!-- <p> En este caso vamos a trabajar con una tabla que contienen información de distintos inmuebles repartidos por una zona específica, sus carácterísticas y su precio </p>  -->

Todas las cuestiones se deben realizar sobre el conjunto de casos que representen viviendas ubicadas en zonas residenciales **(alta, media y baja densidad)**

**MUY IMPORTANTE:** En las otras prácticas he detectado colaboraciones involucrando varias personas y he sido flexible aunque a algunos os lo he mencionado en las correcciones, porque al final el trabajo de analista de datos es un trabajo colaborativo. Sin embargo, este trabajo es individual, así que cuidado con las colaboraciones.

Las variables de las que se compone el dataset son:

|NOMBRE VARIABLE|DESCRIPTOR|VALORES|
| --- | --- | --- |
|Order|Variable de identificación|1 a 2930|
|MS Zoning|Zona de ubicación de la vivienda|"A rural, C comercial, FV residencial flotante, I industrial, RH residencial alta densidad, RL residencial baja densidad, RM residencial media densidad"|
|Lot Frontage|Longitud de la fachada en pies||
|Lot Area|Superficie de la vivienda en pies cuadrados||
|Land Contour|Contorno del terreno circundante|"Lvl llano, Bnk Tipo bancal, HLS Ladera, Low Depresión"|
|Land Slope|Tipo de pendiente de la vivienda|" Gtl pendiente suave, Mod pendiente moderada, Sev fuerte pendiente"|
|Overall Qual|Grado de calidad de materiales y acabado de la vivienda|De 1 (Muy pobre) a 10 (Excelente)|
|Year Built|Año de construccion de la vivienda||
|Year Remod/Add|Año de última reforma de la vivienda||
|Mas Vnr Type|Tipo de revestimiento exterior|" BrkCmn Ladrillo normal, BrkFace Ladrillo visto, CBlock Bloque de cemento, None Ninguna, Stone Piedra "|
|Exter Qual|Calidad de revestimiento exterior|"Ex Excelente,Gd Bueno,TA Media,Fa Justo"|
|Bsmt Cond|Estado general del sótano|"Ex Excelente, Gd Bueno, TA Media, Fa Justo, Po Pobre,Ss sin sótano"|
|Total Bsmt SF|Superficie del sótano en pies cuadrados|
|Heating QC|Calidad de la calefacción|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|Central Air|Aire acondicionado centralizado|"N No Y Sí"|
|Full Bath|Número de baños completo en planta||
|Half Bath|Número de aseos en planta||
|Bedroom AbvGr|Número de dormitorios en planta||
|Kitchen AbvGr|Número de cocinas en planta||
|Kitchen Qual|Calidad de cocinas|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|TotRms AbvGrd|Número total de habitaciones excluidos los cuartos de baño||
|Garage Cars|Número de plazas de garaje||
|Garage Area|Superficie del garaje|||
|Garage Cond|Estado del garaje|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre,Sg sin garaje"|
|Pool Area|Superficie de la piscina en pies cuadrados|
|Pool QC|Calidad de la piscina|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Sp no hay piscina"|
|Mo Sold|mes de venta||
|Yr Sold|año de venta||
|SalePrice|precio de venta en dólares||

Recomiendo al leer los datos, eliminar espacios de los nombres de las columnas, realiza un pequeño análisis inicial de los mismos. No olvides fijarte en los tipos de variables, que variables pueden tener tipos confundidos y corrige los. Sobre todo, trabaja con las fechas.

**NOTA:** Las tareas complementarias sirven para subir nota. El resto de preguntas valen igual y suman 10 puntos.

## Inicializar y cargar el contexto spark

Lo primero que haremos será cargar pyspark y el contexto y las funciones de pyspark-SQL

In [1]:
import pyspark

In [2]:
from pyspark.sql import SQLContext

In [3]:
from pyspark.sql.functions import *

In [4]:
sc = pyspark.SparkContext()

In [5]:
sqlContext = SQLContext(sc)

Procedemos a la carga de datos 

In [6]:
df = sqlContext.read.csv("BDpracticafinalCSV.csv", header=True, sep=';', inferSchema= True)

Cargamos un esquema con la estructora de los datos en el que vemos todas las variables y el tipo de dato. También observamos que existen observaciones nulas en todas las columnas por lo que más adelante tendramos que eliminarlas.

In [7]:
df.printSchema()

root
 |-- Order: integer (nullable = true)
 |-- MS Zoning: string (nullable = true)
 |-- Lot Frontage: integer (nullable = true)
 |-- Lot Area: integer (nullable = true)
 |-- Land Contour: string (nullable = true)
 |-- Land Slope: string (nullable = true)
 |-- Overall Qual: integer (nullable = true)
 |-- Year Built: integer (nullable = true)
 |-- Year Remod/Add: integer (nullable = true)
 |-- Mas Vnr Type: string (nullable = true)
 |-- Exter Qual: string (nullable = true)
 |-- Bsmt Cond: string (nullable = true)
 |-- Total Bsmt SF: integer (nullable = true)
 |-- Heating QC: string (nullable = true)
 |-- Central Air: string (nullable = true)
 |-- Full Bath: integer (nullable = true)
 |-- Half Bath: integer (nullable = true)
 |-- Bedroom AbvGr: integer (nullable = true)
 |-- Kitchen AbvGr: integer (nullable = true)
 |-- Kitchen Qual: string (nullable = true)
 |-- TotRms AbvGrd: integer (nullable = true)
 |-- Garage Cars: integer (nullable = true)
 |-- Garage Area: integer (nullable = tru

Siguiendo lo que dice el enunciado nos quedaremos con las viviendas ubicadas en las zonas residenciales con alta, baja y media densidad. Para ello utilizamos un filter

In [8]:
df = df.filter((df['MS Zoning'] == 'RH') | (df['MS Zoning'] == 'RL') | (df['MS Zoning'] == 'RM'))

Procedemos a eliminar las filas con valores NA. Borramos todos las observaciones con valores nulos y no variable por variable, para que resulte menos tedioso. Lo suyo sería ir variable por variable y borrar o imputar valores a las columnas según cada variable.

In [9]:
df = df.na.drop()

Las variables de tamaño están medidas en **pies** y debido a que mas adelante trabajaremos con **metros** pues vamos a transformar las columnas de pies a metros, teniendo en cuata que **1 pie = 0,093 metros**.
Al realizar esta tranformación la he realizado columna por columna debido a que son pocas, si fueran muchas columnas para cambiar la manera más fácil hubiera sido por medio de una función.

In [10]:
df = df.withColumn("Lot Frontage", df["Lot Frontage"]*0.093)
df = df.withColumn("Lot Area", df["Lot Area"]*0.093)
df = df.withColumn("Total Bsmt SF", df["Total Bsmt SF"]*0.093)
df = df.withColumn("Garage Area", df["Garage Area"]*0.093)
df = df.withColumn("Pool Area", df["Pool Area"]*0.093)

## Cuántas viviendas distintas encontramos en el dataset? ¿Se repite alguna? Tiene sentido que haya duplicadas? ¿Qué podemos hacer con las duplicadas?

Primero contamos las viviendas distintas que hay en el dataset, y depués las totales

In [11]:
df.distinct().count()

2280

In [12]:
df.count()

2282

Como podemos observar, hay 2 viviendas duplicadas por lo tanto como no tiene sentido tener observaciones repetidas, las eliminaremos.

In [13]:
df = df.distinct()

## ¿Podrías decirme el total de inmuebles y el precio medio (Sale Price) de cada zona (MS Zoning)?

Calculamos el total de viviendas como ya lo calculamos antes, pero ahora ya sin observaciones duplicadas

In [14]:
df.count()

2280

Para ver el precio medio en cada zona, lo primero que hay que hacer es agrupar por zonas, y una vez agrupado por zonas obtendremos la media del precio de venta, así como el número de viviendas en cada zona.

In [15]:
df.groupBy('MS Zoning').agg({'SalePrice': 'mean', 'Order':'count'}).show()

+---------+------------+------------------+
|MS Zoning|count(Order)|    avg(SalePrice)|
+---------+------------+------------------+
|       RH|          23|136492.78260869565|
|       RL|        1833| 191550.3109656301|
|       RM|         424|125216.66273584905|
+---------+------------+------------------+



## Media de Total Bsmt SF por cada década de construcción calculada a partir de Year Built. 
## ¿Cuál es la decada de construcción con viviendas mejor acondicionadas para el frío (Heating QC)?

Debido a que las dos preguntas son muy parecidas, he decidido hacerlas las dos juntas para ahorrar tiempo, tambien las he hecho de dos formas diferentes una rápida y otra corta.

Lo primero de todo es hacer un summary de la variable año de construcción, para ver desde que año hasta que año hay datos

In [16]:
df.describe('Year Built').show()

+-------+------------------+
|summary|        Year Built|
+-------+------------------+
|  count|              2280|
|   mean|1969.1429824561403|
| stddev| 31.04392129644678|
|    min|              1872|
|    max|              2010|
+-------+------------------+



Comenzamos haciendo un filtro por cada decada y agrupando por si tienen o no aire acondicionado, además obtendremos la media de metros cuadrados del sótano y contaremos las viviendas con aire y sin aire acondicionado.

In [17]:
df.filter((df['Year Built'] >= 1870) & (df['Year Built'] < 1880)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y|            76.167|                 1|
|          N| 68.07600000000001|                 2|
+-----------+------------------+------------------+



In [18]:
df.filter((df['Year Built'] >= 1880) & (df['Year Built'] < 1890)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y| 72.38057142857143|                 7|
|          N|            115.32|                 1|
+-----------+------------------+------------------+



In [19]:
df.filter((df['Year Built'] >= 1890) & (df['Year Built'] < 1900)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y| 81.69120000000001|                10|
|          N|           85.6995|                 2|
+-----------+------------------+------------------+



In [20]:
df.filter((df['Year Built'] >= 1900) & (df['Year Built'] < 1910)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y| 66.01309090909092|                22|
|          N| 68.48121428571429|                14|
+-----------+------------------+------------------+



In [21]:
df.filter((df['Year Built'] >= 1910) & (df['Year Built'] < 1920)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y|       72.86840625|                64|
|          N| 73.26299999999999|                31|
+-----------+------------------+------------------+



In [22]:
df.filter((df['Year Built'] >= 1920) & (df['Year Built'] < 1930)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y| 79.91961940298508|               134|
|          N| 72.15665853658535|                41|
+-----------+------------------+------------------+



In [23]:
df.filter((df['Year Built'] >= 1930) & (df['Year Built'] < 1940)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y|            73.036|                87|
|          N|          41.73375|                 8|
+-----------+------------------+------------------+



In [24]:
df.filter((df['Year Built'] >= 1940) & (df['Year Built'] < 1950)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y| 72.52735922330099|               103|
|          N|           53.3293|                30|
+-----------+------------------+------------------+



In [25]:
df.filter((df['Year Built'] >= 1950) & (df['Year Built'] < 1960)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y| 90.35375824175829|               273|
|          N|59.581999999999994|                18|
+-----------+------------------+------------------+



In [26]:
df.filter((df['Year Built'] >= 1960) & (df['Year Built'] < 1970)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y| 99.40260714285722|               252|
|          N|115.69199999999998|                 9|
+-----------+------------------+------------------+



In [27]:
df.filter((df['Year Built'] >= 1970) & (df['Year Built'] < 1980)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y| 84.14640000000004|               270|
|          N|            84.816|                 2|
+-----------+------------------+------------------+



In [28]:
df.filter((df['Year Built'] >= 1980) & (df['Year Built'] < 1990)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y| 100.0316551724138|                87|
+-----------+------------------+------------------+



In [29]:
df.filter((df['Year Built'] >= 1990) & (df['Year Built'] < 2000)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y|107.34564406779663|               236|
+-----------+------------------+------------------+



In [30]:
df.filter((df['Year Built'] >= 2000) & (df['Year Built'] < 2010)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y| 128.1739633507853|               573|
|          N|            17.856|                 1|
+-----------+------------------+------------------+



In [31]:
df.filter((df['Year Built'] >= 2010) & (df['Year Built'] < 2020)).groupBy('Central Air') \
.agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+-----------+------------------+------------------+
|Central Air|avg(Total bsmt SF)|count(Central Air)|
+-----------+------------------+------------------+
|          Y|           143.127|                 2|
+-----------+------------------+------------------+



Después de realizar el análisis de cada decada, podemos concluir que la decada de los 2000s en la que más casas acondicionadas hay

Otra forma de hacer lo que hemos hecho antes, será crear una nueva columna en el dataframe con el número de la década, este número lo obtenermos de dividir la cadena de texto Year Built y quedándonos con la 3ª posición.
Este método tiene un problema y es que como tenemos observaciones de viviendas en varios siglos, pues se solaparán los resultados. Pero podría ser una buena aproximación.

In [32]:
split_col = split(df['Year Built'], '')
df = df.withColumn('Dec const', split_col.getItem(2))

In [33]:
df.groupBy('Dec const','Central Air').agg({'Total bsmt SF': 'mean', 'Central Air':'count'}).show()

+---------+-----------+------------------+------------------+
|Dec const|Central Air|avg(Total bsmt SF)|count(Central Air)|
+---------+-----------+------------------+------------------+
|        0|          N|           65.1062|                15|
|        7|          Y| 84.11695571955727|               271|
|        5|          N|59.581999999999994|                18|
|        8|          N|            115.32|                 1|
|        2|          N| 72.15665853658535|                41|
|        3|          N|          41.73375|                 8|
|        9|          Y| 106.3027804878049|               246|
|        1|          Y| 74.99745454545454|                66|
|        2|          Y| 79.91961940298508|               134|
|        9|          N|           85.6995|                 2|
|        4|          N|           53.3293|                30|
|        5|          Y| 90.35375824175829|               273|
|        0|          Y|125.87557815126047|               595|
|       

## ¿Cuáles son las 10 viviendas que se vendieron por un precio más elevado por metro cuadrado en el año 2009?

Para realizar esta pregunta, lo primero que haremos será crear una nueva columna llamada **pxm** que contendrá el precio por metro cuadrado de cada casa

In [34]:
df = df.withColumn("pxm", (df.SalePrice/df['Lot Area']))

Lo siguiente será hacer un filtro para quedarnos con las viviendas vendidas en el año 2009 y las ordenaremos en orden descendente para ver las casas por las que más se ha pagado por metro cuadrado.

In [35]:
df.filter(df['Yr Sold'] == 2009).orderBy('pxm', ascending=False).show(10)

+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+------------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+---------+------------------+
|Order|MS Zoning|Lot Frontage|Lot Area|Land Contour|Land Slope|Overall Qual|Year Built|Year Remod/Add|Mas Vnr Type|Exter Qual|Bsmt Cond|     Total Bsmt SF|Heating QC|Central Air|Full Bath|Half Bath|Bedroom AbvGr|Kitchen AbvGr|Kitchen Qual|TotRms AbvGrd|Garage Cars|Garage Area|Garage Cond|Pool Area|Pool QC|Mo Sold|Yr Sold|SalePrice|Dec const|               pxm|
+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+------------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+---------

## Media anual por zonas del precio de venta y metros cuadrados.

Simplemente tendremos que agrupar por zona y calcular la media del precio de venta y la media de metros cuadrados por vivienda.

In [36]:
df.groupBy('MS Zoning').agg({'SalePrice': 'mean', 'Lot Area':'mean'}).show()

+---------+------------------+-----------------+
|MS Zoning|    avg(SalePrice)|    avg(Lot Area)|
+---------+------------------+-----------------+
|       RH|136492.78260869565|675.3862173913043|
|       RL| 191550.3109656301|991.1073927986907|
|       RM|125216.66273584905|601.1502452830185|
+---------+------------------+-----------------+



## ¿Podrías decirme el total de recaudación de las casas de revistimiento (Mas Vnr Type) de piedra con respecto a las de ladrillo? ¿Hay diferencia significativa?

Creamos un filtro para seleccionar únicamente las casas con revestimiento de piedra o ladrillo y calculamos el total de recaudación.
Como podemos comprobar, el ladrillo liso es el que ha tenido una recaudación casi 3 veces mayor a la de la piedra; y el ladrillo normal es el que menos recaudación ha tenido más de 18 veces menos que la piedra.

In [37]:
df.filter((df['Mas Vnr Type'] == 'BrkCmn')|(df['Mas Vnr Type']=='BrkFace')|(df['Mas Vnr Type']=='Stone')) \
.groupBy('Mas Vnr Type').sum('SalePrice').show()

+------------+--------------+
|Mas Vnr Type|sum(SalePrice)|
+------------+--------------+
|      BrkCmn|       2377175|
|       Stone|      56766127|
|     BrkFace|     139015565|
+------------+--------------+



## ¿Cuánto son más caras las viviendas con 2 cocinas, con 2 o más plazas de garaje que las que tienen 1 cocina y 1 plaza de garaje? Comparar medias y cuartiles de ambos casos

Primero calcularemos los cuatriles y la media de las casas con 2 cocinas y 2 o más garajes. Para ello utilizaremos filter y las funciones para calcular los cuartiles y la media.

In [38]:
df.filter((df['Kitchen AbvGr'] == 2) & (df['Garage Cars'] >= 2)).approxQuantile("SalePrice", [0.25, 0.5, 0.75], 0)

[118858.0, 139500.0, 153337.0]

In [39]:
df.filter((df['Kitchen AbvGr'] == 2) & (df['Garage Cars'] >= 2)).agg({'SalePrice': 'mean'}).show()

+------------------+
|    avg(SalePrice)|
+------------------+
|139113.31944444444|
+------------------+



A continuación calcularemos los cuatriles y la media de las casas con 1 cocina y 1 garaje. 

In [40]:
df.filter((df['Kitchen AbvGr'] == 1) & (df['Garage Cars'] == 1)).approxQuantile("SalePrice", [0.25, 0.5, 0.75], 0)

[110000.0, 127000.0, 140000.0]

In [41]:
df.filter((df['Kitchen AbvGr'] == 1) & (df['Garage Cars'] == 1)).agg({'SalePrice': 'mean'}).show()

+------------------+
|    avg(SalePrice)|
+------------------+
|126712.92424242424|
+------------------+



Podemos observar que claramente las viviendas con 2 cocinas y 2 o más garajes tienen un precio medio y unos cuartiles mayores que las viviendas con 1 cocina y 1 garaje. La conclusión erá la esperada debido a que una casa mejor equipada deberia de costar más.

## (COMPLEMENTARIA) Estudiar la relación entre el precio y el número de Garajes. (Recomiendo segmentar precio por cuartiles y estudiar el número de casos coincidentes)

Para ver la distribución entre el precio y el número de garajes, lo primero que haremos será agrupar por número de garajes y despues obtener la media. Además ordenaremos los garages en orden ascendente para ver como cambia el precio cuando le vamos sumando garajes a la vivienda.
Como resultado vemos que el precio de las viviendas sube a medida que tienen más garajes hasta llegar a 3, momento en el cual empieza a descender su precio.

In [42]:
df.groupBy('Garage Cars').mean('SalePrice').sort(asc("Garage Cars")).show()

+-----------+------------------+
|Garage Cars|    avg(SalePrice)|
+-----------+------------------+
|          0|107568.06569343066|
|          1|126567.36131934033|
|          2| 179271.7467476149|
|          3|318224.63344051444|
|          4|217361.72727272726|
|          5|          126500.0|
+-----------+------------------+



## (COMPLEMENTARIA) Las 10 viviendas con mejores servicios y mejor precio.

Para llevar a cabo este análisis he filtrado por la calidad del revestimiento, sótano, calefacción, cocina, garaje y piscina. He filtrado para que solo aparezacan en las que estas características sean buenas o excelentes. Después he ordenado las viviendas por precio de manera ascendente, mostrándose primero las más barates

In [43]:
df.filter((df['Exter Qual'] == 'Ex')|(df['Exter Qual'] == 'Gd')&(df['Bsmt Cond'] == 'Ex')|(df['Bsmt Cond'] == 'Gd')& \
         (df['Heating QC'] == 'Ex')|(df['Heating QC'] == 'Gd')&(df['Kitchen Qual'] == 'Ex')|(df['Kitchen Qual'] == 'Gd')& \
         (df['Garage Cond'] == 'Ex')|(df['Garage Cond'] == 'Gd')&(df['Pool QC'] == 'Ex')|(df['Pool QC'] == 'Gd')) \
.orderBy('SalePrice', ascending=True).show(10)

+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+---------+------------------+
|Order|MS Zoning|Lot Frontage|Lot Area|Land Contour|Land Slope|Overall Qual|Year Built|Year Remod/Add|Mas Vnr Type|Exter Qual|Bsmt Cond|Total Bsmt SF|Heating QC|Central Air|Full Bath|Half Bath|Bedroom AbvGr|Kitchen AbvGr|Kitchen Qual|TotRms AbvGrd|Garage Cars|Garage Area|Garage Cond|Pool Area|Pool QC|Mo Sold|Yr Sold|SalePrice|Dec const|               pxm|
+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+

### Pista:
#### Calcula las variables: 
- Número de servicios excelentes
- Número de servicios buenos
...

#### Tendréis que tener en cuenta también variables como número de baños Full Bath, cocinas Kitchen AbvGr o dormitorios Bedroom AbvGr. Generando por ejemplo (número de estas variables por encima de media)

#### Finalmente precio de venta